In [1]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-1.5B",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.8, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

INFO 02-27 10:43:09 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.999 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 8.6. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-1.5b-unsloth-bnb-4bit with actual GPU utilization = 73.23%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 12.0 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 7.45 GB. Also swap space = 4 GB.
INFO 02-27 10:44:12 config.py:549] This model supports multiple tasks: {'generate', 'reward', 'embed', 'score', 'classify'}. Defaul

[W227 10:44:18.040962805 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 02-27 10:44:19 loader.py:1089] Loading weights with BitsAndBytes quantization.  May take a while ...
INFO 02-27 10:44:20 weight_utils.py:254] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-27 10:45:44 model_runner.py:1115] Loading model weights took 1.3190 GB
INFO 02-27 10:45:44 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-27 10:45:49 worker.py:267] Memory profiling takes 4.98 seconds
INFO 02-27 10:45:49 worker.py:267] the current vLLM instance can use total_gpu_memory (12.00GiB) x gpu_memory_utilization (0.73) = 8.79GiB
INFO 02-27 10:45:49 worker.py:267] model weights take 1.32GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 1.05GiB; the rest of the memory reserved for KV Cache is 6.39GiB.
INFO 02-27 10:45:49 executor_base.py:111] # cuda blocks: 14957, # CPU blocks: 9362
INFO 02-27 10:45:49 executor_base.py:116] Maximum concurrency for 1024 tokens per request: 233.70x
INFO 02-27 10:45:50 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error o

Capturing CUDA graph shapes: 100%|██████████| 27/27 [00:16<00:00,  1.68it/s]

INFO 02-27 10:46:06 model_runner.py:1562] Graph capturing finished in 16 secs, took 0.84 GiB
INFO 02-27 10:46:06 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 22.10 seconds



Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
tokenizer.chat_template = """{% for message in messages %}{% if message['role'] == 'user' %}{{ '<|im_start|>user\n' + message['content'] + '<|im_end|>\n' }}{% elif message['role'] == 'assistant' %}{{ '<|im_start|>assistant\n' + message['content'] + '<|im_end|>\n' }}{% elif message['role'] == 'system' %}{{ '<|im_start|>system\n' + message['content'] + '<|im_end|>\n' }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"""

In [7]:
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

In [8]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:09<00:00,  9.70s/it, est. speed input: 4.43 toks/s, output: 101.23 toks/s]


"How many s's are in strawberries?猞\n\n猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞猞

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

hf_token = os.getenv("HF_TOKEN")

In [10]:
# model.push_to_hub_merged("yuiseki/YuisekinAI-GRPO-Qwen2.5-1.5B-GSM8K-Lora", tokenizer, save_method = "lora", token = hf_token)